<a href="https://colab.research.google.com/github/Idowuilekura/hamoye_internship/blob/master/HamoyeCalss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import sklearn

In [2]:
hamoye_dataset = pd.read_csv("https://query.data.world/s/wh6j7rxy2hvrn4ml75ci62apk5hgae")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
#Checking distribution of target variable which is qscore
hamoye_dataset.QScore.value_counts()

3A    51481
2A    10576
2B    10096
1A       16
1B       16
Name: QScore, dtype: int64

In [9]:
hamoye_dataset.dropna(inplace=True)

In [10]:
hamoye_dataset.isna().sum()

country           0
year              0
country_code      0
record            0
crop_land         0
grazing_land      0
forest_land       0
fishing_ground    0
built_up_land     0
carbon            0
total             0
QScore            0
dtype: int64

In [11]:
hamoye_dataset.QScore.value_counts()

3A    51473
2A      224
1A       16
Name: QScore, dtype: int64

In [13]:
hamoye_dataset.QScore = hamoye_dataset.QScore.replace(['1A'],'2A')
hamoye_dataset.QScore.value_counts()

3A    51473
2A      240
Name: QScore, dtype: int64

In [16]:
df = hamoye_dataset
df_2A = df[df.QScore=='2A']
df_3A = df[df.QScore=='3A'].sample(350)
data_df = df_2A.append(df_3A)


In [17]:
data_df.QScore.value_counts()

3A    350
2A    240
Name: QScore, dtype: int64

In [22]:
#more preprocessing
data_df = sklearn.utils.shuffle(data_df)
data_df = data_df.reset_index(drop=True)
data_df.shape

(590, 9)

In [23]:
data_df.QScore.value_counts()

3A    350
2A    240
Name: QScore, dtype: int64

In [25]:
X = data_df.drop(columns='QScore')
y= data_df['QScore']

In [33]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test= train_test_split(X,y,test_size=0.3,random_state =0)
y_train.value_counts()

3A    251
2A    162
Name: QScore, dtype: int64

There is still an imbalance in the class distribution, for this we will SMOTE only on the training data to handle this

In [27]:
y_test.value_counts()

3A    99
2A    78
Name: QScore, dtype: int64

In [34]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
x_train.record = label_encoder.fit_transform(x_train.record)
x_test.record = label_encoder.fit_transform(x_test.record)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [35]:
import imblearn 
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state =1)
x_train_balanced,y_balanced = smote.fit_sample(x_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [38]:
print(type(x_train_balanced))

<class 'numpy.ndarray'>


In [45]:
pd.Series(y_balanced).value_counts()

3A    251
2A    251
dtype: int64

In [46]:
x_train_balanced = pd.DataFrame(x_train_balanced,columns=x_train.columns)

In [48]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalised_train_df = scaler.fit_transform(x_train_balanced.drop(columns=['record']))
normalised_train_df =pd.DataFrame(normalised_train_df,columns=x_train_balanced.drop(['record'],axis=1).columns)
normalised_train_df['record'] = x_train_balanced['record']
x_test = x_test.reset_index(drop=True)
normalised_test_df = scaler.transform(x_test.drop(columns=['record']))
normalised_test_df = pd.DataFrame(normalised_test_df,columns=x_test.drop(columns=['record']).columns)
normalised_test_df['record'] = x_test['record']

In [50]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(normalised_train_df,y_balanced)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Measuring Classification Performance 

In [51]:
# cross-validation and accuracy
from sklearn.model_selection import cross_val_score
scores =cross_val_score(log_reg,normalised_train_df,y_balanced,cv=5,scoring='f1_macro')
scores

array([0.5390873 , 0.52433281, 0.54887218, 0.54959464, 0.63869932])

In [52]:
from sklearn.model_selection import KFold, StratifiedKFold, LeaveOneOut
from sklearn.metrics import recall_score,accuracy_score,precision_score,f1_score

In [55]:
#k-fold cross validation
from sklearn.model_selection import KFold
kf= KFold(n_splits=5)
kf.split(normalised_train_df)
f1_scores =[]
for train_index,test_index in kf.split(normalised_train_df):
  x_train_kfold,x_test_kfold=normalised_train_df.iloc[train_index],normalised_train_df.iloc[test_index]
  y_train_kfold,y_test_kfold = y_balanced[train_index],y_balanced[test_index]
  model = LogisticRegression().fit(x_train_kfold,y_train_kfold)
  f1_scores.append(f1_score(y_test_kfold,model.predict(x_test_kfold),pos_label='2A')*100)


In [68]:
from sklearn.model_selection import KFold 
f1_scores =[]
kf = KFold(n_splits=5)
for train_index,test_index in kf.split(normalised_train_df):
  x_train_fold,x_test_fold = normalised_train_df.iloc[train_index],normalised_train_df.iloc[test_index]
  y_train_fold,y_test_fold = y_balanced[train_index],y_balanced[test_index]
  model = sklearn.linear_model.LogisticRegression().fit(x_train_fold,y_train_fold)
  f1_scores.append(f1_score(y_test_fold,model.predict(x_test_fold),pos_label='2A')*100)

In [73]:
# stratified k_fold cross validation
from sklearn.model_selection import StratifiedKFold
sk = StratifiedKFold(n_splits=5,shuffle=True,random_state=1)
scores = []
for train_index,test_index in sk.split(normalised_train_df,y_balanced):
  x_train_sk,x_test_sk = normalised_train_df.iloc[train_index],normalised_train_df.iloc[test_index]
  y_train_sk,y_test_sk= y_balanced[train_index],y_balanced[test_index]
  model = sklearn.linear_model.LogisticRegression().fit(x_train_sk,y_train_sk)
  f1_scores.append(f1_score(y_test_sk,model.predict(x_test_sk),pos_label='2A')*100,)

Leave one out cross validation (LOOCV)

In [79]:
from sklearn.model_selection import LeaveOneOut
loo= LeaveOneOut()
score = cross_val_score(LogisticRegression(),normalised_train_df,y_balanced,cv=loo,scoring='f1_macro')
average_score = score.mean()
average_score

0.5617529880478087

In [81]:
from sklearn.metrics import confusion_matrix

In [84]:
new_predictions = log_reg.predict(normalised_test_df)
cnf_mat = confusion_matrix(y_true=y_test,y_pred=new_predictions ,labels=['2A','3A'])
cnf_mat

array([[38, 40],
       [49, 50]])

In [86]:
# Accuracy 
accuracy = accuracy_score(y_true=y_test,y_pred=new_predictions)
accuracy

0.4971751412429379

In [88]:
#precision 
precision = precision_score(y_test,new_predictions,pos_label='2A')
precision


0.4367816091954023

In [89]:
#recall 
recall = recall_score(y_test,new_predictions,pos_label='2A')
recall

0.48717948717948717

In [91]:
#f1_score 
f1_score = f1_score(y_test,new_predictions,pos_label='2A')
f1_score

0.46060606060606063